# Bark Experiments 
### Using Transformers library


___

## Initial Setup

In [16]:
# Import dependencies
from transformers import AutoProcessor, BarkModel
from IPython.display import Audio
import torch
import scipy

In [17]:
# Setup accelerator device (CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [18]:
# Define parameters - precision and model type
torch_dtype = torch.float32 # Default 
model_type = "suno/bark"  # Default. Alternative model is "suno/bark-small"

In [19]:
# Instantiate processor and Bark model
processor = AutoProcessor.from_pretrained(model_type, torch_dtype=torch_dtype)
model = BarkModel.from_pretrained(model_type, torch_dtype=torch_dtype)

C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\transformers\models\encodec\modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [20]:
# Shift model to accelerator device (CUDA)
model = model.to(device)
model.device

device(type='cuda', index=0)

In [21]:
# View model architecture details
model

BarkModel(
  (semantic): BarkSemanticModel(
    (input_embeds_layer): Embedding(129600, 1024)
    (position_embeds_layer): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x BarkBlock(
        (layernorm_1): BarkLayerNorm()
        (layernorm_2): BarkLayerNorm()
        (attn): BarkSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (att_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): BarkMLP(
          (in_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
          (gelu): GELU(approximate='none')
        )
      )
    )
    (layernorm_final): BarkLayerNorm()
    (lm_head): Linear(in_feat

In [22]:
# Define voice preset (optional)
voice_preset = "v2/en_speaker_6"

# # Define text prompt 
# text_prompt = '''
# [clears throat] Is this the real life? Is this just fantasy? [laughs]
# Caught in a landslide, no escape from reality!
# '''
text_prompt=input()

___
## Generate Audio Output

In [23]:
# Pass voice preset and text prompt into processor object
inputs = processor(text=text_prompt, voice_preset=voice_preset)

# View input audio tensors
inputs

{'input_ids': tensor([[91327, 13028, 48628, 10142, 10531, 46267, 10112, 11859, 19478, 12131,
           117, 10319, 28005, 19777, 10134, 10105, 10422, 10464, 10108, 17122,
         13877, 10108, 10105, 62375,   106,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
# Generate output audio arrays (tensors) from Bark model
audio_arrays = model.generate(**inputs.to(device))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [13]:
# Convert output arrays into NumPy arrays, remove dimensions of size 1 from shape of array, and shift to CPU
audio_arrays = audio_arrays.cpu().numpy().squeeze()

In [14]:
# Set sampling rate (Default is 24kHz)
sample_rate = model.generation_config.sample_rate

In [15]:
# Convert audio output into sampled audio to be played in notebook widget
Audio(audio_arrays, rate=sample_rate)

In [ ]:
# Download audio output as wav file
# scipy.io.wavfile.write("bark_out.wav", rate=sample_rate, data=audio_arrays)

___
## Exploration of Bark's Capabilities with Prompt Engineering

In [ ]:
# Create function to generate audio array
def generate_audio(text_prompt, voice_preset=None):
    inputs = processor(text=text_prompt, voice_preset=voice_preset)
    audio_arrays = model.generate(**inputs.to(device)).cpu().numpy().squeeze()
    return audio_arrays

### (1) Multilingual Speech Synthesis

#### (1.1) Mandarin

In [ ]:
text_prompt = '千里之行，始于足下。所以，不要害怕迈出第一步'

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

In [ ]:
text_prompt = '千里之行，始于足下。所以，不要害怕迈出第一步'

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

#### (1.2) Hindi

In [ ]:
text_prompt = 'धैर्य और समय, दोनों ही बहुत महत्वपूर्ण हैं। जब तुम अपने लक्ष्य की ओर बढ़ रहे हो, तो इन दोनों की आवश्यकता होती है। धैर्य रखो और समय दो, सफलता जरूर मिलेगी।'

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

___
### (2) Code Switching

#### (2.1) French - English

In [ ]:
text_prompt = '''
Mon père me disait toujours: 
"The best way to predict the future is to create it."
'''

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

___
### (3) Non-Verbal Effects

#### Mix of non-verbal effects

In [ ]:
text_prompt = '''
Remember when we tried cooking that 5-minute recipe, but it took us THREE HOURS? [laughs]
And the kitchen looked like ... a war zone afterwards! [sighs]
'''

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

#### Speaker Prompts
(Does not consistently give the correct output)

In [ ]:
text_prompt = '''
WOMAN: I would like an oatmilk latte please.
MAN: Wow, that's expensive!
'''

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

___
### (4) Music

In [ ]:
text_prompt = '♪ I want to break free! I want to break FREE!!  ♪'

audio_arrays = generate_audio(text_prompt)
Audio(audio_arrays, rate=model.generation_config.sample_rate)

___